# ***Hi, This is the continuation of [this](https://www.kaggle.com/ironben/rdbs-to-graphdb-neo4j-network-approach/) data transformation kernel, with EDA, NLP and the Recommendation Engine*** 
*Feel free to ask questions and upvote it if you think it is useful for you.*

### Answering Jared's questions:

Did you decide to predict Pros given a Question, or predict Questions given a Pro? Why?
*I did the former because it's more suitable for the email notification systome, however if I turn around the query it can recommend unanswered questions for a given professional real time.*
Does your model address the "cold start" problem for Professionals who just signed up but have not yet answered any questions? How'd you approach that challenge?
*Not directly, however witht the Graph database and the Cypher query language it could be solved by Jaccard score and closeness in the graph*
Did your model have any novel approaches to ensuring that "no question gets left behind"?
*The unanswered questions could be queried real time to build a burn down board for the CV site.*
What types of models did you try, and why did you pick the model architecture that you picked?
I considered a few approaches (ML, NLP, networkx, Neo4j), but the Neo4j graph approach turned out the best for the current tactical needs and future strategis directions
Did you engineer any new data features? Which ones worked well for your model?
The Jaccard score was great to find out similar questions, with more tags (for NLP or from users) it could be even more precise.
Is there anything built into your approach to protect certain Professionals from being "overburdened" with a disproportionately high share of matches?
*No, but it could be done easily by a query builded into the recommendation system.*
What do you wish you could use in the future (that perhaps we could try out on our own as an extension of your work)?
*The graph based link predicton ML approach, the book got out a few days ago...*

Import the files for later for the NLP tokenisation:

In [ ]:
import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from wordcloud import WordCloud,STOPWORDS
import warnings
warnings.filterwarnings('ignore')
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from IPython.display import Image


In [ ]:
answers = pd.read_csv('../input/data-science-for-good-careervillage/answers.csv')
comments = pd.read_csv('../input/data-science-for-good-careervillage/comments.csv')
emails = pd.read_csv("../input/data-science-for-good-careervillage/emails.csv")
group_memberships = pd.read_csv('../input/data-science-for-good-careervillage/group_memberships.csv')
groups = pd.read_csv('../input/data-science-for-good-careervillage/groups.csv')
matches = pd.read_csv('../input/data-science-for-good-careervillage/matches.csv')
professionals = pd.read_csv("../input/data-science-for-good-careervillage/professionals.csv")
questions = pd.read_csv('../input/data-science-for-good-careervillage/questions.csv')
school_memberships = pd.read_csv('../input/data-science-for-good-careervillage/school_memberships.csv')
students = pd.read_csv('../input/data-science-for-good-careervillage/students.csv')
tag_questions = pd.read_csv("../input/data-science-for-good-careervillage/tag_questions.csv")
tag_users = pd.read_csv('../input/data-science-for-good-careervillage/tag_users.csv')
tags = pd.read_csv('../input/data-science-for-good-careervillage/tags.csv')

In [ ]:
df_list = [answers,comments,emails,group_memberships,groups,matches,professionals,questions,school_memberships,students,tag_questions,tag_users,tags]

***EDA***

In [ ]:
Image(filename="../input/cv-graph-schema2/graph_schema_kaggle.png")

* Old schema (up)
* New DB schema (down) after a few modification showed in the previous kernel:
*Location of students and professionals and the industry of a professional where extracted into nodes.*


In [ ]:
Image(filename="../input/temp-pics/new_schema.png")

**How many nodes whe have?**

> MATCH (n) RETURN COUNT(n) AS CountOfNodes

╒══════════════╕
│"CountOfNodes"│
╞══════════════╡
│2027427       │
└──────────────┘


**Relationships?**

> MATCH ()-[m]->() RETURN COUNT(m) AS CountOfRels

╒═════════════╕
│"CountOfRels"│
╞═════════════╡
│6619257      │
└─────────────┘

**How many node labels we have grouped by type?**

> CALL db.labels()
YIELD label
CALL apoc.cypher.run("MATCH (:"+`label`+") RETURN count(*) as count", null)
YIELD value
RETURN label, value.count AS count
ORDER BY count DESC

label	count
* "Email"	1850101
* "Answer"	51123
* "Student"	30971
* "Professional"	28152
* "Question"	23931
* "Tag"	16269
* "Comment"	14966
* "Location"	6682
* "School"	2706
* "Industry"	2470
* "Group"	49
* "Group_Type"	7
* "Matches"	0

**How many relationships we have grouped by type?**

> CALL db.relationshipTypes()
YIELD relationshipType
CALL apoc.cypher.run("MATCH ()-[:"+`relationshipType` +"] -> () RETURN count(*) as count", null)
YIELD value
RETURN relationshipType, value.count AS count
ORDER BY count DESC

relationshipType	count
* "IS_IN"	4316275
* "GOT_EMAIL"	1850101
* "HAS_TAG"	212460
* "AUTHOR_OF"	88264
* "IS_REPLY_TO"	66089
* "BASED_IN"	53992
* "WORKING_IN"	25576
* "MEMBER_IN"	6451
* "HAS_TYPE"	49

One questions tags are connecting other questions, professionals and student to each other:
*Red: Student
Dust: Professional
DarkBlue: Question
LightBlue: Tag*


In [ ]:
Image(filename="../input/temp-pics/question_tags.png")

We could go deeper, and get lost in the graph but I would like to take and iterative approach to build the recommendation engine and explore the graph along the path.

***Approach One to Recommendation Engine*** Get similar questions (with answers) to the student's unanswered questions, because this is THE fastes way to get answers to the students because it only depends on past professional activity and do not need future one, in most cases, ***I call this as a first line of help.***

**Check out the unanswered questions:**

> MATCH (q:Question)
WHERE not exists ((q)-[:IS_REPLY_TO]-(:Answer))
WITH q
MATCH (q)-[:HAS_TAG]-(t:Tag)
return  q._id AS question_id, count(t) as tag_c order by tag_c desc

question_id	tag_c
* "5194c9cac7134c868beb02f83c1a5bbf"	14
* "9ee85a6f4ab14a878928871ab3e86616"	10
* "72affe86909d49c09761a1d376476c92"	9
* "f4790c8029064b41bd410f66c5039890"	9
* "c7cb6c54dbb34c0ab2eb23751deea6ad"	9
* "785f2a8c69bb4f9787fa87a4d487014f"	9
* "fd8c6ff37c0d4439980a04d9d3ec5ffe"	9
* "4189060e1882495fb4af4cdd7447d074"	8
* "00a1ac3d65094cba96d9a719ace341d9"	8
* ....
* "dab7b240dc394d30a54dd0c5862d5fe3" 3
* ...

**The dataset has 807 unanswered questions, and the db had given this result in 154 ms**



Check out these questions, the "5194c9cac7134c868beb02f83c1a5bbf" with 14 tags and the "dab7b240dc394d30a54dd0c5862d5fe" one with 3 tags.

> MATCH (q:Question {_id:"dab7b240dc394d30a54dd0c5862d5fe3"})-[r]-(k) return q,r,k

In [ ]:
print(os.listdir("../input/temp-pics"))

In [ ]:
Image(filename="../input/temp-pics/dab_q_tag.png")

As you can see an unanswered question on has 3 types of nodes in the neighbourhood, the tags, the email and the author, our best path is on the tags.

> MATCH (q:Question {_id:"5194c9cac7134c868beb02f83c1a5bbf"})-[r:HAS_TAG]-(t:Tag) return q,r,t limit 300

In [ ]:
Image(filename="../input/temp-pics/q_519_tag.png")

**Get similar answers based on Jaccard index for the question with 3 tags:**
> MATCH (q_or:Question {_id:"dab7b240dc394d30a54dd0c5862d5fe3"})-[:HAS_TAG]-(t:Tag)-[:HAS_TAG]-(q_ot:Question)-[IS_REPLY_TO]-(a:Answer)
WITH q_or, q_ot, COUNT(t) AS intersection, COLLECT(t.name) AS i
MATCH (q_or)-[:HAS_TAG]-(o_or:Tag)
WITH q_or, q_ot, intersection,i, COLLECT(o_or.name) AS s1
MATCH (q_ot)-[:HAS_TAG]-(o_ot:Tag)
WITH q_or, q_ot,intersection,i, s1, COLLECT(o_ot.name) AS s2

WITH q_or, q_ot,intersection,s1,s2

WITH q_or, q_ot,intersection,s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN q_or.title, q_ot.title, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC LIMIT 20


* **Original question title: "Are there any resume items which would make my college application really stand out?"**
* Most similar question: "How do you make your university application stand out?"
* Second: "Can someone proofread my college transfer essay?"
* Third: "What should I look for if I am considering transferring colleges?"
* Fourth: "How do you build your resume in college?"
* Fifth: "How did you decide which college to attend?"


So it is quite powerful, considering it is only a query in a db with 2M nodes and 6M relationships and had given these results in 211 ms so it has real time applications

**Get similar answers based on Jaccard index for the question with 14 tags:**
> MATCH (q_or:Question {_id:"5194c9cac7134c868beb02f83c1a5bbf"})-[:HAS_TAG]-(t:Tag)-[:HAS_TAG]-(q_ot:Question)-[IS_REPLY_TO]-(a:Answer)
WITH q_or, q_ot, COUNT(t) AS intersection, COLLECT(t.name) AS i
MATCH (q_or)-[:HAS_TAG]-(o_or:Tag)
WITH q_or, q_ot, intersection,i, COLLECT(o_or.name) AS s1
MATCH (q_ot)-[:HAS_TAG]-(o_ot:Tag)
WITH q_or, q_ot,intersection,i, s1, COLLECT(o_ot.name) AS s2
WITH q_or, q_ot,intersection,s1,s2
WITH q_or, q_ot,intersection,s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2
RETURN q_or.title, q_ot.title, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC LIMIT 20



* **Original question title: "What are the best free resources for MCAT prep?"**
* Most similar question: "How have you reacted to the people who have told you that you don't have what it takes to become a doctor/surgeon?"
* Second: "What helps you study for a test?"
* Third: "How do I decide between career paths?"
* Fourth: "What is the most rewarding part about being a doctor?"
* Fifth: "What would you tell a high school/college student who has goals of attending Medical School but isn't sure what to specialize in?"

It was a bit slower 1139ms and you can build a recommendation function for a push of a button.

**Simpler solution and example with the 3 tags:**
> MATCH (q_o:Question {_id:"dab7b240dc394d30a54dd0c5862d5fe3"})-[:HAS_TAG]-(t:Tag)-[:HAS_TAG]-(q_s:Question)-[IS_REPLY_TO]-(a:Answer)-[:IS_REPLY_TO]-(c:Comment)
RETURN q_o.title AS Question_WithOut_Answer, q_s.title AS Similar_Question, a.body AS Answers, count(distinct t) as tag_count, count(distinct c) as comment_count ORDER BY tag_count desc,comment_count desc limit 5

* **Original question title: "Are there any resume items which would make my college application really stand out?"**
* Most similar question:"When applying to universities, the people at admissions always enjoy reading unconventional essays. How does one know that what they are writing fits this criteria?"
* Second:"Can someone proofread my college transfer essay?"

***Approach Two to Recommendation Engine*** recommend a professional who had answered questions with similar tags

Question with 3 tags:
> MATCH (q_o:Question {_id:"dab7b240dc394d30a54dd0c5862d5fe3"})-[:HAS_TAG]-(t:Tag)-[:HAS_TAG]-(q_s1:Question)-[:IS_REPLY_TO]-(a:Answer)-[:AUTHOR_OF]-(p:Professional)-[:AUTHOR_OF]-(a_o:Answer)-[:IS_REPLY_TO]-(q_s2:Question)-[:HAS_TAG]-(:Tag)-[:HAS_TAG]-(q_o)
RETURN q_o.title AS Question_WithOut_Answer, p._id AS Professional, count(distinct t) as tag_count, count(distinct a_o) as p_a_count ORDER BY tag_count desc, p_a_count desc limit 5

|Professional |  tag_count | p_a_count  | 
|---|---|---|
|"36ff3b3666df400f956f8335cf53e09e"|3          |36         |
|"05ab77d4c6a141b999044ebbf5415b0d"|2          |23     |
|"a1006e6a58a0447592e2435caa230f78"|2          |8        |
|"bb4fe08bb5424f08972927d90bad9642"|2          |5          |
|"369f1c8646b649f6997eae7809696bd5"|2          |5          |

Question with 3 tags:
> MATCH (q_o:Question {_id:"dab7b240dc394d30a54dd0c5862d5fe3"})-[:HAS_TAG]-(t:Tag)-[:HAS_TAG]-(q_s1:Question)-[:IS_REPLY_TO]-(a:Answer)-[:AUTHOR_OF]-(p:Professional)-[:AUTHOR_OF]-(a_o:Answer)-[:IS_REPLY_TO]-(q_s2:Question)-[:HAS_TAG]-(:Tag)-[:HAS_TAG]-(q_o)
RETURN q_o.title AS Question_WithOut_Answer, p._id AS Professional, count(distinct t) as tag_count, count(distinct a_o) as p_a_count ORDER BY tag_count desc, p_a_count desc limit 5

|Professional |  tag_count | p_a_count  | 
|---|---|---|
|"36ff3b3666df400f956f8335cf53e09e"|9          |134        |
|"05ab77d4c6a141b999044ebbf5415b0d"|8      |61         |
|"0355aaad83704ad98e6d71b79db85849"|8      |19         |
|"a1006e6a58a0447592e2435caa230f78"|7        |94         |
|"a6d33c38902546849c36ea7e9e9f0870"|7         |52         |


As you can see the best improvement opportunity is to upgrade the tagging system.

**The whole recommendation engine's idea was based on this system:**
[https://neo4j.com/sandbox-v2/#](https://neo4j.com/sandbox-v2/#) look for the recommendation engine sandbox. Moreover you can understand why we need the hearts :)

This was it for today, thanks for following it through, I will continue the improvement :)

TODO for recommendation improvement: 
* NLP based question tagging for more connections,
* professional activity dates in the query to filter out inactive professionals
* use heart data 
* NLP based answer tagging for answer recommendation for an unanswered question ?
* student get todolist data from competition owners


I'm doing the tag enrichment based on this [kernel](https://www.kaggle.com/danielbecker/careervillage-org-recommendation-engine), to improve the connection density and improve the recommendation's quality.

# NLP based tag enrichment !RUN OUT OF TIME TO DO THIS PROPERLY!

In [ ]:
# Merge Question Title and Body
questions['questions_full_text'] = questions['questions_title'] +'\r\n\r\n'+ questions['questions_body']

In [ ]:
import spacy
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

# Spacy Tokenfilter for part-of-speech tagging
token_pos = ['NOUN', 'VERB', 'PROPN', 'ADJ', 'INTJ', 'X']

def nlp_preprocessing(data):
    """ Use NLP to transform the text corpus to cleaned sentences and word tokens

    """    
    def token_filter(token):
        """ Keep tokens who are alphapetic, in the pos (part-of-speech) list and not in stop list

        """    
        return not token.is_stop and token.is_alpha and token.pos_ in token_pos
    
    processed_tokens = []
    data_pipe = nlp.pipe(data)
    for doc in data_pipe:
        filtered_tokens = [token.lemma_.lower() for token in doc if token_filter(token)]
        processed_tokens.append(set(filtered_tokens))
    return processed_tokens

# Get NLP Tokens
questions['nlp_tokens'] = nlp_preprocessing(questions['questions_full_text'])

In [ ]:
questions['nlp_tokens'].head()

### Removing token which are more than x% of the questions

In [ ]:
all_q_tokens = list(questions['nlp_tokens'])

In [ ]:
type(all_q_tokens)

In [ ]:
all_unique_q_tokens = set([token for token_list in all_q_tokens for token in token_list])
flat_q_tokens = [token for token_list in all_q_tokens for token in token_list]

In [ ]:
len(all_q_tokens)

In [ ]:
all_q_tokens[:5]

In [ ]:
len(all_unique_q_tokens)

In [ ]:
list(all_unique_q_tokens)[:5]

In [ ]:
len(flat_q_tokens)

In [ ]:
flat_q_tokens[:5]

In [ ]:
from collections import Counter,OrderedDict

In [ ]:
tokens_sum = Counter(flat_q_tokens)
sorted_ord_tokens = sorted(tokens_sum.items(), key=lambda kv: kv[1],reverse=True)

In [ ]:
sorted_ord_tokens

# Import tags into database a rerun the tests

### WIP, you can use this code to generate mora tags based on the questions, and answer tags.

USING PERIODIC COMMIT 10000 
LOAD CSV WITH HEADERS FROM "file:///q_exp_id_and_nlp_tokens.csv"AS row
MATCH (q:Question {_id:row.questions_id})
WITH SPLIT(row.nlp_tokens, ",") AS tags,q,row
UNWIND tags as tag
	CREATE (t:Token:Tag {name:tag})
	CREATE (q)-[:HAS_TAG]->(t)

CREATE INDEX ON :Tag(name)
CREATE INDEX ON :Token(name)

MATCH (n:Tag)
WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count
WHERE count > 1
CALL apoc.refactor.mergeNodes(nodelist) YIELD node
RETURN node


In [ ]:
import spacy
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

# Spacy Tokenfilter for part-of-speech tagging
token_pos = ['NOUN', 'VERB', 'PROPN', 'ADJ', 'INTJ', 'X']

def nlp_preprocessing(data):
    """ Use NLP to transform the text corpus to cleaned sentences and word tokens

    """    
    def token_filter(token):
        """ Keep tokens who are alphapetic, in the pos (part-of-speech) list and not in stop list

        """    
        return not token.is_stop and token.is_alpha and token.pos_ in token_pos
    
    processed_tokens = []
    data_pipe = nlp.pipe(data)
    for doc in data_pipe:
        filtered_tokens = [token.lemma_.lower() for token in doc if token_filter(token)]
        temp_string = ','
        processed_tokens.append(temp_string.join(list(set(filtered_tokens))))
    return processed_tokens

# Get NLP Tokens
questions['nlp_tokens'] = nlp_preprocessing(questions['questions_full_text'])

In [ ]:
questions['nlp_tokens'].head()

In [ ]:
q_exp = questions[['questions_id','nlp_tokens']]

In [ ]:
q_exp.to_csv("q_exp_id_and_nlp_tokens.csv", index=False)

In [ ]:
q_exp.head().to_csv("q_exp_test_5.csv", index=False)